In [1]:
data_dir = "C:/Users/Imper/Downloads/state-farm-distracted-driver-detection/imgs/train"

In [2]:
batch_size = 32

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(
    rescale=1/255,
    validation_split=0.2) # set validation split)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        data_dir,  # This is the source directory for training images
        target_size=(640, 480),  # All images will be resized to 200 x 200
        batch_size=batch_size,
        # Specify the classes explicitly
        classes = ['c0','c1','c2','c3','c4','c5','c6','c7','c8','c9'],
        # Since we use categorical_crossentropy loss, we need categorical labels
        class_mode='categorical',
        subset='training')

test_generator = train_datagen.flow_from_directory(
        data_dir,  # This is the source directory for training images
        target_size=(640, 480),  # All images will be resized to 200 x 200
        batch_size=batch_size,
        # Specify the classes explicitly
        classes = ['c0','c1','c2','c3','c4','c5','c6','c7','c8','c9'],
        # Since we use categorical_crossentropy loss, we need categorical labels
        class_mode='categorical',
        subset='validation')

Found 17943 images belonging to 10 classes.
Found 4481 images belonging to 10 classes.


In [3]:
import keras
keras.__version__
from keras import models
from keras import layers

In [4]:
def get_my_model ():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(640, 480, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    return model

In [5]:
model = get_my_model()
model.summary()
model.compile(optimizer='Nadam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 638, 478, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 319, 239, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 317, 237, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 158, 118, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 156, 116, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 78, 58, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 76, 56, 64)        3

In [6]:
import pandas as pd

#Save Model and history
def save_model(history,model):
  model.save("/content/drive/My Drive/Colab Notebooks/models/")
  hist_df = pd.DataFrame(history.history) 
  hist_df.to_csv('data.csv')
  !cp data.csv "/content/drive/My Drive/Colab Notebooks/"
  return

In [7]:
def create_new_model(model):
  history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = test_generator, 
    validation_steps = test_generator.samples // batch_size,
    epochs = 6)
  save_model(history,model)
  return (history, model)

In [8]:
#Load a model
def load_model():
  model = models.load_model("C:/Users/Imper/Downloads/model31")
  history = pd.read_csv("C:/Users/Imper/Downloads/model31/data.csv")
  return (history,model)

In [9]:
#Create a new model
#(history)(model) = create_new_model(model)

#Load a trained model
history_model = load_model()
history = history_model[0]
model = history_model[1]

In [10]:
import matplotlib.pyplot as plt

plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
#plt.ylim([0.9,1])
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(history['loss'])
plt.plot(history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
#plt.ylim([0,.4])
plt.legend(['train','test'], loc='upper left')
plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [11]:
keras.utils.plot_model(model,"model.png",show_shapes = True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [12]:
import numpy as np
from keras.preprocessing import image

test_image = image.load_img("C:/Users/Imper/Downloads/bb1.jpg", target_size = (640, 480))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

result = model.predict(test_image)
print(result)

[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]


In [13]:
#test_loss, test_acc = model.evaluate(X_Test, Y_Test, verbose = 1)